<a href="https://colab.research.google.com/github/ankile/Adversarial-Diffusion/blob/main/src/Adverserial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Adversarial training

Code to train adversarially robust models as part of the diffusion as adversarial defense project.

The following two cells must be run before restarting the runtime to ensure that the `tiatoolbox` package loads correctly.


In [ ]:
%%capture
!pip install tiatoolbox    

In [ ]:
%%capture
!apt-get -y install libopenjp2-7-dev libopenjp2-tools openslide-tools

In [ ]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from tiatoolbox.models.architecture import get_pretrained_model
from torchvision.datasets import pcam
from torchvision import transforms
from tiatoolbox.models.dataset.classification import predefined_preproc_func
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

f"Running on: {device}"

'Running on: cuda'

### Load Datasets

The PCam dataset from TiaToolBox was used. A subset of the validation set was used for training, as it was smaller than the training set, and therefore did not take up precious memory.


In [ ]:
# load preprocessing
preprocess_pcam = predefined_preproc_func("pcam")

# load total datasets
train_dataset = pcam.PCAM(
    root="data", split="val", download=True, transform=preprocess_pcam.func
)

# load test data
test_dataset = pcam.PCAM(
    root="data", split="test", download=True, transform=preprocess_pcam.func
)

### Adversarial Example Creator

The following function generates adversarial examples from a given image, for a given model. This was used to create training examples during adversarial training, and in testing across all methods.


In [ ]:
def create_adverserial(input_tensor, true, model, no_iter=5, lr=1e5):
    epsilon = 2 / 255
    # epsilon = 2/255/10
    delta = torch.zeros_like(input_tensor, requires_grad=True)
    opt = optim.Adam([delta], lr=lr)

    for t in range(no_iter):
        pred = model((input_tensor + delta).to(device).unsqueeze(0)).to(device)
        loss = -nn.CrossEntropyLoss()(
            pred.to(device), torch.LongTensor([true]).to(device)
        )

        opt.zero_grad()
        loss.backward()
        opt.step()
        delta.data.clamp_(-epsilon, epsilon)
    return input_tensor + delta

### Adversarial Training & Test function

**Adversarial Training:** Function that loops over training data, creating adversarial examples and updating model parameters.

**Test**: Function that loops over test data, creating adversarial examples and measuring both standard and robust accuracy.


In [ ]:
def adverserial_training(net, trainloader, lr=1e-5, epoch_no=1):
    print(f"learning rate:{lr}")
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=lr)

    for epoch in range(epoch_no):  # loop over the dataset multiple times
        running_loss = 0.0

        for i, (inputs, labels) in tqdm(
            enumerate(trainloader, 0), total=len(trainloader)
        ):
            # get the inputs; data is a list of [inputs, labels]
            inputs = inputs.to(device)
            labels = labels.to(device)

            # change model status
            net.eval()

            # create adverserial example
            inputs = create_adverserial(inputs.squeeze(0), labels, net).unsqueeze(0)

            # change model status
            net.train()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    print("Finished Training")
    return net


def test(net, test_dataset, test_size=500):
    # 1. Create Adverserial Examples
    subset_test_dataset = torch.utils.data.Subset(
        test_dataset, list(range(0, test_size))
    )
    testloader = torch.utils.data.DataLoader(
        subset_test_dataset,
        batch_size=1,
    )

    # change model status
    net.eval()

    correct_std = 0
    total_std = 0
    # create test_dataset & calculate standard accuracy
    test_adverserial_set = []
    for i, (inputs, labels) in tqdm(enumerate(testloader, 0), total=len(testloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_std += labels.size(0)
        correct_std += (predicted == labels).sum().item()

        # create adverserial example
        adverserial = create_adverserial(inputs.squeeze(0), labels, net)
        test_adverserial_set.append([adverserial, labels])

    print(
        f"Standard accuracy of the network on the {test_size} test images: {100 * correct_std // total_std} %"
    )
    std_accuracy = 100 * correct_std // total_std

    # 2. Test Model
    net.eval()
    correct = 0
    total = 0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for inputs, labels in test_adverserial_set:
            # calculate outputs by running images through the network
            outputs = net(inputs.unsqueeze(0))
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(
        f"Adversarial Accuracy of the network on the {test_size} test images: {100 * correct // total} %"
    )
    accuracy = 100 * correct // total

    return accuracy, std_accuracy
    # return std_accuracy

### Adversarial Training of 3 Different Classifiers

In the following code, we load seven pre-trained models and perform adversarial training on each of them, from which we determined that GoogLeNet performed the best.


In [ ]:
models = [
    "resnet101",
    "resnet50",
    "googlenet",
    "wide_resnet101_2",
    "alexnet",
    "densenet201",
    "resnext101",
]

for model_selection in models:
    print(f"Using model: {model_selection}")
    model = get_pretrained_model(pretrained_model=f"{model_selection}-pcam")
    net = model[0].to(device)

    # sample random training data
    train_indices = np.random.choice(
        np.arange(len(train_dataset)), size=1000, replace=False
    )
    # create data loader from sample subset
    subset_train_dataset = torch.utils.data.Subset(train_dataset, train_indices)
    trainloader = torch.utils.data.DataLoader(
        subset_train_dataset,
        batch_size=1,
        shuffle=True,
    )

    # adverserial training
    net = adverserial_training(net, trainloader, lr=1e-10)
    accuracy, std_accuracy = test(net, test_dataset, test_size=1000)

Using model: wide_resnet101_2
Download from https://tiatoolbox.dcs.warwick.ac.uk/models/pc/wide_resnet101_2-pcam.pth
Save to /root/.tiatoolbox/models/wide_resnet101_2-pcam.pth
learning rate:1e-10


100%|██████████| 100/100 [00:31<00:00,  3.17it/s]


Finished Training


100%|██████████| 1000/1000 [04:02<00:00,  4.12it/s]


Standard accuracy of the network on the 1000 test images: 51 %
Adversarial Accuracy of the network on the 1000 test images: 51 %


### Adversarial Training Using Varying Number of Training Examples

Next we analyzed how the performance changed depending on the number of adversarial examples used during training.


In [ ]:
# points to stop at
i = [1, 10, 50, 100, 500, 1000]

# load starting model
model_selection = models[1]
model = get_pretrained_model(pretrained_model=f"{model_selection}-pcam")
net = model[0]
net.to("cuda")

# accuracy scores
acc_scores = []

for j in range(len(i)):
    if j == 0:
        index_start = 0
    else:
        index_start = i[j - 1] + 1
    index_end = i[j]

    # select subset of data
    subset_train_dataset = torch.utils.data.Subset(
        train_dataset, list(range(index_start, index_end))
    )
    trainloader = torch.utils.data.DataLoader(
        subset_train_dataset,
        batch_size=1,
    )

    # adverserial training
    net = adverserial_training(net, trainloader)

    # test
    accuracy = test(net, test_dataset, test_size=100)
    acc_scores.append(accuracy)